<a href="https://colab.research.google.com/github/fjgreco/LLM/blob/main/LLMHacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install torch

In [10]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.7 MB/s eta 0:00:00


In [11]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [12]:
!pip install optimum
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
  Created wheel for optimum: filename=optimum-1.13.2-py3-none-any.whl size=395599 sha256=e719ee4e57c1c4f1d23fc89eda393803c4167d9d1da9b46cfe53792938b68655
  Stored in directory: /root/.cache/pip/wheels/6e/b7/2c/7

In [16]:
!pip install wikipedia

In [1]:
from transformers import AutoConfig, AutoModel

from transformers import AutoModelForCausalLM

from transformers import AutoTokenizer

import torch

import accelerate

In [2]:
mn='TheBloke/Llama-2-7b-Chat-GPTQ'
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors='pt')
toks['input_ids']

tensor([[    1,  5677,  6764, 17430,   338,   263, 29871]])

In [4]:
model=AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ',device_map=0, torch_dtype=torch.float16)

In [5]:
res=model.generate(toks['input_ids'].to('cuda'),max_new_tokens=30).to('cpu')
#res

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [6]:
tokr.batch_decode(res)

['<s> Jeremy Howard is a 35-year-old man from the United States who was arrested in Thailand for allegedly attempting to smuggle 1.3 kil']

In [2]:
import wikipedia
jh_page=wikipedia.summary("Coronavirus")

In [3]:
len(jh_page.split())

158

In [4]:
def gen(prompt, maxlen=15, model='', sample=True):
    toks = tokr(prompt, return_tensors='pt',)
    print(toks)
    res=model.generate(**toks.to('cuda'), max_new_tokens=maxlen, do_sample=sample).to('cpu')

In [5]:
def mk_prompt(user, syst=''): return f"{syst}###\n\n### Assistant:\n"

In [6]:
ques = "What is coronavirus"

ques_ctx = f"""Answer the question with the help of the provided context.

## Context

{jh_page}

## Question

{ques}"""

In [8]:
mn="stabilityai/StableBeluga-7B"
tokr = AutoTokenizer.from_pretrained(mn)

In [9]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extrememly well. Help as much as you can.\n\n"

In [11]:
model=AutoModelForCausalLM.from_pretrained(mn,device_map=0, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
res=gen(mk_prompt(ques_ctx,syst=sb_sys),model=model,maxlen=300)

{'input_ids': tensor([[    1,   835,  2184, 29901,    13,  3492,   526,   624,   519,  3741,
         26281, 29892,   385,   319, 29902,   393,  4477, 11994,  9413,   331,
           368,  1532, 29889, 22305,   408,  1568,   408,   366,   508, 29889,
            13,    13,  2277, 29937,    13,    13,  2277, 29937,  4007, 22137,
         29901,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
print(res)

None
